In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from pandas.plotting import table
from tqdm import tqdm
from dataclasses import dataclass
from scipy.stats import norm
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
import time
import random as rnd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
from sklearn import preprocessing
%matplotlib inline
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from collections import Counter
from imblearn.over_sampling import SMOTE

In [ ]:
pip install --upgrade xlrd

In [ ]:
viz = pd.read_excel('default of credit card clients.xls')
viz.columns = viz.iloc[0]
viz = viz.drop(viz.index[0]).rename(columns = {'default payment next month': 'DEFAULT/NO-DEFAULT'})
viz = viz.apply(pd.to_numeric)
viz

In [ ]:
cred = pd.read_excel('default of credit card clients.xls')
cred.columns = cred.iloc[0]
cred = cred.drop(cred.index[0]).rename(columns = {'default payment next month': 'DEFAULT/NO-DEFAULT'})
cred = cred.apply(pd.to_numeric)
cred.set_index('ID', inplace=True)

In [ ]:
cred

In [ ]:
limit = cred.pop("LIMIT_BAL")
age = cred.pop("AGE")

In [ ]:
cred.insert(loc = 9, column = 'LIMIT_BAL', value=limit)
cred.insert(loc = 10,  column = 'AGE', value = age)

In [ ]:
cred.dtypes

In [ ]:
fil = (cred.EDUCATION == 5) | (cred.EDUCATION == 6) | (cred.EDUCATION == 0)
cred.loc[fil, 'EDUCATION'] = 4
cred.EDUCATION.value_counts()

In [ ]:
cred.loc[cred.MARRIAGE == 0, 'MARRIAGE'] = 3
cred.MARRIAGE.value_counts()

In [ ]:
cred.isna().sum()

In [ ]:
cred.describe()

In [ ]:
pd.DataFrame(cred.apply(lambda col: col.unique()))

In [ ]:
duplicate = cred[cred.duplicated()]
duplicate

## VISUALIZATIONS

In [ ]:
numeric_df = viz[['BILL_AMT1', 	  'BILL_AMT2',	'BILL_AMT3',	'BILL_AMT4',	'BILL_AMT5',	'BILL_AMT6',	'PAY_AMT1',	'PAY_AMT2',	'PAY_AMT3',	'PAY_AMT4',	'PAY_AMT5',	'PAY_AMT6']]
correlation = numeric_df.corr()
fig, ax = plt.subplots(figsize=(20,10))   
sns.heatmap(correlation, xticklabels=correlation.columns,yticklabels=correlation.columns,cmap="RdBu",square=True,annot=True )

In [ ]:
df_temp1 = viz.groupby(['EDUCATION', 'SEX'])['ID'].count().reset_index()
df_temp1.columns = ['source', 'target', 'value']
df_temp1['source'] = df_temp1.source.map({0: "Missing1", 1 : "Graduate", 2: "University", 3 : "High School", 4 : "Other1", 5 : "Other2", 6 : "Other3"})
df_temp1['target'] = df_temp1.target.map({1 : 'Male', 2 : 'Female'})
df_temp2 = viz.groupby(['SEX', 'MARRIAGE'])['ID'].count().reset_index()
df_temp2.columns = ['source', 'target', 'value']
df_temp2['source'] = df_temp2.source.map({1 : 'Male', 2 : 'Female'})
df_temp2['target'] = df_temp2.target.map({0 : 'Missing2', 1 : 'Married', 2 : 'Single', 3 : 'Other'})
df_temp3 = viz.groupby(['MARRIAGE', 'DEFAULT/NO-DEFAULT'])['ID'].count().reset_index()
df_temp3.columns = ['source', 'target', 'value']
df_temp3['source'] = df_temp3.source.map({0 : 'Missing2', 1 : 'Married', 2 : 'Single', 3 : 'Other'})
df_temp3['target'] = df_temp3.target.map({1 : 'Default', 0 : 'Not Default'})
links = pd.concat([df_temp1, df_temp2, df_temp3], axis = 0)
unique_source_target = list(pd.unique(links[['source', 'target']].values.ravel("K")))
mapping_dict = {k : v for v, k in enumerate(unique_source_target)}
links['source'] = links['source'].map(mapping_dict)
links['target'] = links['target'].map(mapping_dict)
links_dict = links.to_dict(orient = 'list')
fig = go.Figure(data = [go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = 'black', width = 0.5),
        label = unique_source_target,
        color = 'blue'
        ),
    
    link = dict(
        source = links_dict['source'],
        target = links_dict['target'],
        value = links_dict['value']
    )
    
)]
)
fig.update_layout(title_text = 'Default of Credit Card Clients Alluvial Diagram', font_size = 10)
fig.show()

In [ ]:
df2 = viz
df2['April'] = df2['BILL_AMT6'] - df2['PAY_AMT6']
df2['May'] = df2['BILL_AMT5'] - df2['PAY_AMT5']
df2['June'] = df2['BILL_AMT4'] - df2['PAY_AMT4']
df2['July'] = df2['BILL_AMT3'] - df2['PAY_AMT3']
df2['August'] = df2['BILL_AMT2'] - df2['PAY_AMT2']
df2['September'] = df2['BILL_AMT1'] - df2['PAY_AMT1']
df2 = df2[['SEX','April', 'May', 'June', 'July', 'August', 'September']]
df2 = df2.groupby(by = ['SEX']).sum()
df2 = df2.T
df2 = pd.DataFrame(df2).reset_index()
# df2.reset_index()
df2.plot(x = 0, kind='barh', stacked=True,
        title='Difference between billed and payed amounts across gender in April-September', mark_right = True)

plt.show()

In [ ]:
bins = [10, 25, 35, 50, 75, 100]
viz['AGE'] = pd.cut(viz['AGE'], bins = bins)
viz
df3 = viz.groupby(['AGE']).apply(lambda x: x.sort_values(['LIMIT_BAL'], ascending=False).head())

df3['EDUCATION'] = df3.EDUCATION.map({0: "Missing1", 1 : "Graduate", 2: "University", 3 : "High School", 4 : "Other1", 5 : "Other2", 6 : "Other3"})
df3['SEX'] = df3.SEX.map({1 : 'Male', 2 : 'Female'})
df3['MARRIAGE'] = df3.MARRIAGE.map({0 : 'Missing2', 1 : 'Married', 2 : 'Single', 3 : 'Other'})


fig = px.treemap(df3, path=['AGE', 'EDUCATION', 'SEX'], values = "LIMIT_BAL", color = "LIMIT_BAL")
fig.show()

In [ ]:
viz.columns

In [ ]:
BOX = viz.boxplot(column = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6'],
                  by ='DEFAULT/NO-DEFAULT', figsize =(15,15))

In [ ]:
HEX = viz.plot.hexbin(x='DEFAULT/NO-DEFAULT', y='LIMIT_BAL', gridsize=10);

In [ ]:
df_a = viz
df_copy1 = df_a
df_1 = df_copy1[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
                'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]
df_statement = df_1[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']]
df_statement = df_statement.rename(columns={"BILL_AMT1": "September", "BILL_AMT2": "August", "BILL_AMT2": "July", 
                                            "BILL_AMT4": "June", "BILL_AMT5": "May", "BILL_AMT6": "April"})
df_statement_sum = df_statement.sum(axis=0)
df_statement_sum = pd.DataFrame(df_statement_sum)
df_paid = df_1[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]
df_paid = df_paid.rename(columns={"PAY_AMT1": "September", "PAY_AMT2": "August", "PAY_AMT3": "July", 
                                  "PAY_AMT4": "June", "PAY_AMT5": "May", "PAY_AMT6": "April"})
df_paid_sum = df_paid.sum(axis=0)
df_paid_sum = pd.DataFrame(df_paid_sum)
df_result = pd.merge(df_statement_sum, df_paid_sum, left_index=True, right_index=True)
df_result = df_result.rename(columns={"0_x": "Total Statement Amount", "0_y": "Total Paid Amount"})
df_result.plot(kind='bar')
plt.show()

## MODEL IMPLEMENTATION

TAKING RANDOM SAMPLING OF THE DATASET AS DECISION TREE WAS TAKING ~6 HOURS TO RUN AND NAIVE BAYES WAS TAKING ~ 40 MINS TO RUN.

In [ ]:
cred_1 = cred.sample(n = 10000)
cred_1

In [ ]:
cred_1

## BASELINE MODEL

In [ ]:
y = cred_1.iloc[:, -1].to_numpy()
X = cred_1.drop(['DEFAULT/NO-DEFAULT'], axis = 1)

In [ ]:
X_1, X_2, y_1, y_2 = train_test_split(X, y, random_state = 0, test_size = 0.3)

In [ ]:
import statistics

from sklearn.metrics import classification_report

estm = statistics.mode(y_1)

y_pred = [estm] * len(y_2)

print(classification_report(y_2, y_pred))

### SPLITTING DATASET INTO X AND Y

In [ ]:
y = cred_1.iloc[:, -1].to_numpy()
X = cred_1.drop(['DEFAULT/NO-DEFAULT'], axis = 1)

TRAIN_TEST SPLIT + NORMALIZATION OF NUMERICAL COLUMNS

In [ ]:
# y = cred_1.iloc[:, -1].to_numpy()
# X = cred_1.drop(['DEFAULT/NO-DEFAULT'], axis = 1)
# len(y)

In [ ]:
@dataclass
class algo:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def numericalnorm(self, X_train, X_test):
        self.X_1 = self.X_train.iloc[:, 0:9] # Xtrain not normalize
        self.X_2 = self.X_train.iloc[:, 9:23] # Xtrain normalize
        self.X_11 = self.X_test.iloc[:, 0:9] # Xtest not normalize
        self.X_22 = self.X_test.iloc[:, 9:23] # Xtest normalize

        
        return self.X_1, self.X_2, self.X_11, self.X_22
#         print(X_11)
#         print(X_12)
        
    def dataSplit(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.3, random_state = 0)
#         print(X_train)
        
        return self.X_train, self.X_test, self.y_train, self.y_test
        
    def normalizeTrainX(self, X):
        
        mean = np.mean(X, 0)
        std = np.std(X, 0)
        normalized = (X - mean)/std
        return normalized, mean, std
#         print(normalized)

    def normalizeTestX(self, X, trainMean, trainStd):
        normalized = (X - trainMean)/trainStd
        return normalized
#         print(trainMean)
 
    def datafit(self, X, y):

        X_train, X_test, y_train, y_test = self.norm()


        return X_train, X_test, y_train, y_test
        
    
    def norm(self):
        X_train, X_test, y_train, y_test = self.dataSplit(self.X, self.y)
        
        sm = SMOTE(random_state = 0)
        self.X_train, self.y_train = sm.fit_resample(self.X_train, self.y_train)
        
        X_1, X_2, X_11, X_22 = self.numericalnorm(self.X_train, self.X_test)
        


        X_train, mean, std = self.normalizeTrainX(self.X_2)        
        X_train1 = pd.concat([self.X_1, X_train], axis = 1)
        
        X_test1 = self.normalizeTestX(self.X_22, mean, std)
        
        X_test1 = pd.concat([self.X_11, X_test1], axis = 1)
        
#         print(len(self.y_test))
        return X_train1, X_test1, self.y_train, y_test


        

        
        




In [ ]:
al = algo(X,y)
# al.numericalnorm()
# al.dataSplit(X_2, y)
al.datafit(X, y)

## Logistic Regression

In [ ]:
@dataclass
class LogisticRegression:
    
    def __init__(self, learningRate, tolerance, maxIteration) -> None:
        self.tolerance = tolerance
        self.learningRate = learningRate
        self.maxIteration = maxIteration
        self.d = None

  
    def sigmoid(self, z):
        sig = 1/(1+np.exp(-z))
        return sig

    def costFunction(self, X, y):
        sig = self.sigmoid(X.dot(self.w))
        # approach1
        cost = (y*np.log(sig)+(1-y)*np.log(1-sig)).sum()
        cost = -cost/X.shape[0]
        # approach2
        # cost = (np.log(np.ones(X.shape[0])+np.exp(X.dot(self.w))-X.dot(self.w).dot(y)))
        # cost = cost.sum()
        return cost

    def gradient(self, X, y):
        sig = self.sigmoid(X.dot(self.w))
        gradient = (sig-y).dot(X)
        return gradient

    def gradientDescent(self, X, y):
        errors = []
        last_error = float('inf')
        
        for i in tqdm(range(self.maxIteration)):
            self.w = self.w - self.learningRate * self.gradient(X, y)
            current_error = self.costFunction(X, y)
            diff = last_error - current_error
            last_error = current_error
            errors.append(current_error)

            if np.abs(diff) < self.tolerance:
                print('model stopped learning')
                break
        return

    def predict(self, X):
        sig = self.sigmoid(X.dot(self.w))
        return np.around(sig)               

    def evaluate(self, y, y_hat):
        y = (y==1)
        y_hat = (y_hat == 1)                
        recall = (y & y_hat).sum()/y.sum()
        precision = (y & y_hat).sum()/y_hat.sum()
        f_score = 2*(precision*recall)/(precision+recall)
        accuracy = np.sum(y == y_hat)/len(y)
        print(y_hat)
        
        df = {
        
        'Accuracy': [accuracy],
        'F1-Score':  [f_score],
        'Recall': [recall],
        'Precision': [precision]}

        self.d = pd.DataFrame(df)
        
        return f_score, recall, precision, accuracy

    def fit(self):
        X_train, X_test, y_train, y_test = al.datafit(X, y)
#         print(X_test)
        print(X_train.shape[1])
        print('Solving using GD')
        self.w = np.ones(X_train.shape[1], dtype=np.float64)*0
        self.gradientDescent(X_train, y_train)
        print(self.w)
        print(X_test.columns)
        y_test_hat = self.predict(X_test)
        f_score, recall, precision, accuracy = self.evaluate(y_test, y_test_hat)
        print('Training F1 score {}'.format(f_score))
        print('Training recall score {}'.format(recall))
        print('Training precision score {}'.format(precision))
        print('Accuray {}'.format(accuracy))
        
        
        
        
        
        fpr, tpr, _ = roc_curve(y_test, y_test_hat)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=[5,5])
        plt.plot(fpr,tpr, color = 'darkorange', lw=2, label = 'ROC curve (area = %0.4f)' % roc_auc)
        plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.legend(loc='lower right')
        plt.show()
        
        confusion_matrix = metrics.confusion_matrix(y_test, y_test_hat)
        cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

        cm_display.plot()
        plt.show()
        
        


In [ ]:
lr1 = LogisticRegression(learningRate = 0.01, tolerance = 0.05, maxIteration = 10000)

In [ ]:
%%time
lr1.fit()

In [ ]:
%%time
lr2 = LogisticRegression(learningRate = 0.001, tolerance = 0.005, maxIteration = 50000)
lr2.fit()

In [ ]:
%%time
lr3 = LogisticRegression(learningRate = 0.0001, tolerance = 0.0005, maxIteration = 50000)
lr3.fit()

In [ ]:
%%time
lr4 = LogisticRegression(learningRate = 0.00001, tolerance = 0.00005, maxIteration = 50000)
lr4.fit()

In [ ]:
%%time
lr5 = LogisticRegression(learningRate = 0.00001, tolerance = 0.0005, maxIteration = 50000)
lr5.fit()

In [ ]:
l1 = pd.DataFrame(lr1.d)
l2 = pd.DataFrame(lr2.d)
l3 = pd.DataFrame(lr3.d)
l4 = pd.DataFrame(lr4.d)
l5 = pd.DataFrame(lr5.d)
logistic = pd.concat([l1, l2, l3, l4, l5])
logistic = logistic.reset_index(drop = True)
logistic['Learning Rate'] = [0.01, 0.001, 0.0001, 0.00001, 0.00001]
logistic['Tolerance'] = [0.05, 0.005, 0.0005, 0.00005, 0.0005]
logistic


In [ ]:
logistic_best = pd.DataFrame(lr3.d)
logistic_best

## Naive Bayes

In [ ]:
@dataclass
class NaiveBayes:
    X:np.matrix
    y:np.array

    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.d = None

    def dataSplit(self):
        self.X_train, self.X_test, self.y_train, self.y_test = al.datafit(X, y)

    def fitDistribution(self, data):
        mean = np.mean(data)
        std = np.std(data)
        dist = norm(mean, std)
        return dist

    def probabilityFunction(self, X, prior, dist1, dist2, dist3, dist4, dist5, dist6, dist7, dist8, dist9, dist10, dist11,
                           dist12, dist13, dist14, dist15, dist16, dist17, dist18, dist19, dist20, dist21, dist22, dist23):
        return prior*dist1*dist2*dist3*dist4*dist5*dist6*dist7*dist8*dist9*dist10.pdf(X[9])*dist11.pdf(X[10])*dist12.pdf(X[11])*dist13.pdf(X[12])*dist14.pdf(X[13])*dist15.pdf(X[14])*dist16.pdf(X[15])*dist17.pdf(X[16])*dist18.pdf(X[17])*dist19.pdf(X[18])*dist20.pdf(X[19])*dist21.pdf(X[20])*dist22.pdf(X[21])*dist23.pdf(X[22])
          
    
    def discreteNaive(self, X, y):
        
        a = 1
        X_train = X
        y_train = y
        k = len(np.unique(self.X))
        
        if (np.where(X_train)[0].shape[0]/len(y_train))==0:
            return (np.where(X_train)[0].shape[0]+a)/(len(y_train)+a*k)
        else:
            # else return the prob
            return np.where(X_train)[0].shape[0]/len(y_train)
        

    def fit(self):
        
        self.dataSplit()

        print(self.X_train.shape, self.y_train.shape)
        self.X0_train = self.X_train[self.y_train == 0]
        self.X1_train = self.X_train[self.y_train == 1]

        self.prior_y0 = len(self.X0_train)/len(self.X_train)
        self.prior_y1 = len(self.X1_train)/len(self.X_train)
        
        #Continuous variables

        self.dist_X0_9 = self.fitDistribution(self.X0_train.values[:, 9]) #Gaussian probability for LIMIT_BAL of class 0
        self.dist_X0_10 = self.fitDistribution(self.X0_train.values[:, 10]) #Gaussian probability for AGE of class 0
        self.dist_X0_11 = self.fitDistribution(self.X0_train.values[:, 11]) #Gaussian probability for BILL_AMT1 of class 0
        self.dist_X0_12 = self.fitDistribution(self.X0_train.values[:, 12]) #Gaussian probability for BILL_AMT2 of class 0
        self.dist_X0_12 = self.fitDistribution(self.X0_train.values[:, 12]) #Gaussian probability for BILL_AMT2 of class 0
        self.dist_X0_13 = self.fitDistribution(self.X0_train.values[:, 13]) #Gaussian probability for BILL_AMT3 of class 0
        self.dist_X0_14 = self.fitDistribution(self.X0_train.values[:, 14]) #Gaussian probability for BILL_AMT4 of class 0
        self.dist_X0_15 = self.fitDistribution(self.X0_train.values[:, 15]) #Gaussian probability for BILL_AMT5 of class 0
        self.dist_X0_16 = self.fitDistribution(self.X0_train.values[:, 16]) #Gaussian probability for BILL_AMT6 of class 0
        self.dist_X0_17 = self.fitDistribution(self.X0_train.values[:, 17]) #Gaussian probability for PAY_AMT1 of class 0
        self.dist_X0_18 = self.fitDistribution(self.X0_train.values[:, 18]) #Gaussian probability for PAY_AMT2 of class 0
        self.dist_X0_19 = self.fitDistribution(self.X0_train.values[:, 19]) #Gaussian probability for PAY_AMT3 of class 0
        self.dist_X0_20 = self.fitDistribution(self.X0_train.values[:, 20]) #Gaussian probability for PAY_AMT4 of class 0
        self.dist_X0_21 = self.fitDistribution(self.X0_train.values[:, 21]) #Gaussian probability for PAY_AMT5 of class 0
        self.dist_X0_22 = self.fitDistribution(self.X0_train.values[:, 22]) #Gaussian probability for PAY_AMT6 of class 0
        self.dist_X1_9 = self.fitDistribution(self.X1_train.values[:, 9]) #Gaussian probability for LIMIT_BAL of class 1
        self.dist_X1_10 = self.fitDistribution(self.X1_train.values[:, 10]) #Gaussian probability for AGE of class 1
        self.dist_X1_11 = self.fitDistribution(self.X1_train.values[:, 11]) #Gaussian probability for BILL_AMT1 of class 1
        self.dist_X1_12 = self.fitDistribution(self.X1_train.values[:, 12]) #Gaussian probability for BILL_AMT2 of class 1
        self.dist_X1_13 = self.fitDistribution(self.X1_train.values[:, 13]) #Gaussian probability for BILL_AMT3 of class 1
        self.dist_X1_14 = self.fitDistribution(self.X1_train.values[:, 14]) #Gaussian probability for BILL_AMT4 of class 1
        self.dist_X1_15 = self.fitDistribution(self.X1_train.values[:, 15]) #Gaussian probability for BILL_AMT5 of class 1
        self.dist_X1_16 = self.fitDistribution(self.X1_train.values[:, 16]) #Gaussian probability for BILL_AMT6 of class 1
        self.dist_X1_17 = self.fitDistribution(self.X1_train.values[:, 17]) #Gaussian probability for PAY_AMT1 of class 1
        self.dist_X1_18 = self.fitDistribution(self.X1_train.values[:, 18]) #Gaussian probability for PAY_AMT2 of class 1
        self.dist_X1_19 = self.fitDistribution(self.X1_train.values[:, 19]) #Gaussian probability for PAY_AMT3 of class 1
        self.dist_X1_20 = self.fitDistribution(self.X1_train.values[:, 20]) #Gaussian probability for PAY_AMT4 of class 1
        self.dist_X1_21 = self.fitDistribution(self.X1_train.values[:, 21]) #Gaussian probability for PAY_AMT5 of class 1
        self.dist_X1_22 = self.fitDistribution(self.X1_train.values[:, 22]) #Gaussian probability for PAY_AMT6 of class 1

    def predict(self):
        y_pred = []
        
        for sample, target in zip(self.X_test.values, self.y_test):
            
            py0 = self.probabilityFunction(sample, self.prior_y0, 
                                           self.discreteNaive(self.X0_train.values[:, 0]==sample[1], self.X0_train),
                                           self.discreteNaive(self.X0_train.values[:, 1]==sample[1], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 2]==sample[2], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 3]==sample[3], self.X0_train),
                                           self.discreteNaive(self.X0_train.values[:, 4]==sample[1], self.X0_train),
                                           self.discreteNaive(self.X0_train.values[:, 5]==sample[5], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 6]==sample[6], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 7]==sample[7], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 8]==sample[8], self.X0_train), 
                                           self.dist_X0_9, self.dist_X0_10, 
                                           self.dist_X0_11, self.dist_X0_12, self.dist_X0_13, 
                                           self.dist_X0_14, self.dist_X0_15, self.dist_X0_16, self.dist_X0_17, 
                                           self.dist_X0_18,self.dist_X0_19, self.dist_X0_20, self.dist_X0_21, 
                                           self.dist_X0_22)
            py1 = self.probabilityFunction(sample, self.prior_y1, 
                                           self.discreteNaive(self.X1_train.values[:, 0]==sample[1], self.X1_train),
                                           self.discreteNaive(self.X1_train.values[:, 1]==sample[1], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 2]==sample[2], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 3]==sample[3], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 4]==sample[1], self.X1_train),
                                           self.discreteNaive(self.X1_train.values[:, 5]==sample[5], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 6]==sample[6], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 7]==sample[7], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 8]==sample[8], self.X1_train), 
                                           self.dist_X1_9, self.dist_X1_10,
                                           self.dist_X1_11, self.dist_X1_12, self.dist_X1_13, 
                                           self.dist_X1_14, self.dist_X1_15, self.dist_X1_16, self.dist_X1_17, 
                                           self.dist_X1_18,self.dist_X1_19, self.dist_X1_20, self.dist_X1_21, 
                                           self.dist_X1_22)
        
            y_pred.append(np.argmax([py0, py1]))
          
        y_pred = np.array(y_pred)  
        
        print(y_pred)
        
        
        accuracy = np.sum(self.y_test == y_pred)/len(self.y_test)
        precision = (self.y_test & y_pred).sum()/y_pred.sum()
        recall = (self.y_test & y_pred).sum() / self.y_test.sum()
        f_score = 2 * (precision * recall) / (precision + recall)
        
        print("Accuracy is {}".format(accuracy)) 
        print("F-score is {}".format(f_score)) 
        print("Precision is {}".format(precision))
        print("Recall is {}".format(recall))
        
        df = {       
        'Accuracy': [accuracy],
        'F1-Score':  [f_score],
        'Recall': [recall],
        'Precision': [precision]}

        self.d = pd.DataFrame(df)
        
        fpr, tpr, _ = roc_curve(self.y_test, y_pred)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=[5,5])
        plt.plot(fpr,tpr, color = 'darkorange', lw=2, label = 'ROC curve (area = %0.4f)' % roc_auc)
        plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.legend(loc='lower right')
        plt.show()
        
        confusion_matrix = metrics.confusion_matrix(self.y_test, y_pred)
        cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

        cm_display.plot()
        plt.show()

In [ ]:
clf = NaiveBayes(X, y)

In [ ]:
%%time
clf.fit()

In [ ]:
%%time
clf.predict()

In [ ]:
naive = pd.DataFrame(clf.d)
naive

## Decision Tree

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [ ]:
class DecisionTreeClassifier():
    def __init__(self, X, y, min_samples_split = 2, max_depth = 2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

        self.X = X
        self.y = y
        self.d = None
        self.flag = True
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, y):
        ''' function to compute leaf node '''
        
        y = list(y)
        return max(y, key=y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        if self.flag:
          print("Decision Tree Representation :\n")    
          self.flag=False
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def evaluate(self, X_test, y):
        y_hat = self.predict(self.X_test) 
        y_hat = np.array(y_hat).reshape(len(y_hat),1)
        # print(y_hat)
        y = (y==1)
        y_hat = (y_hat == 1)
        accuracy = np.sum(y == y_hat)/len(y)
        recall = (y & y_hat).sum()/y.sum()
        precision = (y & y_hat).sum()/y_hat.sum()
        f_score = 2*(precision*recall)/(precision+recall)
        print("Metric Report :\nF1-Score:{}\nRecall:{}\nPrecision:{}\nAccuracy:{}".format(f_score, recall, precision, accuracy))
        
        fpr, tpr, _ = roc_curve(self.y_test, y_hat)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=[5,5])
        plt.plot(fpr,tpr, color = 'darkorange', lw = 2, label = 'ROC curve (area = %0.4f)' % roc_auc)
        plt.plot([0,1], [0,1], color='navy', lw = 2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.legend(loc='lower right')
        plt.show()
        
        confusion_matrix = metrics.confusion_matrix(self.y_test, y_hat)
        cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

        cm_display.plot()
        plt.show()
        
        df = {
        'Accuracy': [accuracy],
        'F1-Score':  [f_score],
        'Recall': [recall],
        'Precision': [precision]}

        self.d = pd.DataFrame(df)
        
        
   
        
    def fit(self):
        ''' function to train the tree '''
        from sklearn.model_selection import train_test_split
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
        dataset = np.column_stack((self.X_train, self.y_train))
        self.root = self.build_tree(dataset)
        y_pred = self.evaluate(self.X_test, self.y_test)
        # print("Decision Tree Representation :\n")
        # self.print_tree()
#         return X_train, X_test, y_train, y_test, y_pred
    


    def info(self):
        self.fit()
        self.print_tree()
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [ ]:
X = cred_1.iloc[:, :-1].values
y = cred_1.iloc[:, -1].values.reshape(-1,1)

In [ ]:
%%time
dt5 = DecisionTreeClassifier(X, y, min_samples_split = 8, max_depth = 4)
dt5.info()

In [ ]:
dec_tree5 = pd.DataFrame(dt5.d)
dec_tree5

## COMPARISON

In [ ]:
analysis = pd.concat([logistic_best, naive, dec_tree5])
analysis = analysis.reset_index(drop = True)
analysis

In [ ]:
analysis['Model'] = ['Logistic', 'Naive', 'Decision Tree']
analysis

In [ ]:
import plotly.graph_objects as go

models = ['Logistic', 'Naive Bayes', 'Decision Tree']

fig = go.Figure()
fig.add_trace(go.Bar(x = models,
                y = analysis['Accuracy'],
                name ='Accuracy',
                marker_color='salmon'
                ))
fig.add_trace(go.Bar(x = models,
                y = analysis['F1-Score'],
                name='F1-Score',
                marker_color='sandybrown'
                ))
fig.add_trace(go.Bar(x = models,
                y=analysis['Recall'],
                name='Recall',
                marker_color='saddlebrown'
                ))
fig.add_trace(go.Bar(x=models,
                y=analysis['Precision'],
                name='Precision',
                marker_color='lightsalmon'
                ))

fig.update_layout(
    title='Comparison of Machine Learning Models',
    xaxis_tickfont_size=14,
    yaxis = dict(
        title='Evaluation Scores',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x = 1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)',
        yanchor="top",
        xanchor = "right",
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

### ACCURACY ANALYSIS

In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'Accuracy_test' :  analysis['Accuracy']

})

models.sort_values(by = 'Accuracy_test', ascending = False)
px.bar(data_frame = models, y = 'Accuracy_test', x = 'Model',
       color = 'Accuracy_test',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'Accuracy Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")

### F1-SCORE ANALYSIS

In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'F1-Score' :  analysis['F1-Score']

})

models.sort_values(by = 'F1-Score', ascending = False)
px.bar(data_frame = models, y = 'F1-Score', x = 'Model',
       color = 'F1-Score',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'F1-Score Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")


### RECALL ANALYSIS

In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'Recall' :  analysis['Recall']

})

models.sort_values(by = 'Recall', ascending = False)
px.bar(data_frame = models, y = 'Recall', x = 'Model',
       color = 'Recall',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'Recall Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")


### PRECISION ANALYSIS

In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'Precision' :  analysis['Precision']

})

models.sort_values(by = 'Precision', ascending = False)
px.bar(data_frame = models, y = 'Precision', x = 'Model',
       color = 'Precision',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'Precision Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")


## FEATURE ENGINEERING STEP

In [ ]:
cred_2 = cred
cred_2.dtypes

In [ ]:
NET_AMT1 = cred_2['BILL_AMT1'] - cred_2['PAY_AMT1']
NET_AMT2 = cred_2['BILL_AMT2'] - cred_2['PAY_AMT2']
NET_AMT3 = cred_2['BILL_AMT3'] - cred_2['PAY_AMT3']
NET_AMT4 = cred_2['BILL_AMT4'] - cred_2['PAY_AMT4']
NET_AMT5 = cred_2['BILL_AMT5'] - cred_2['PAY_AMT5']
NET_AMT6 = cred_2['BILL_AMT6'] - cred_2['PAY_AMT6']

In [ ]:
cred_2.insert(11, 'NET_AMT1', NET_AMT1)
cred_2.insert(12, 'NET_AMT2', NET_AMT2)
cred_2.insert(13, 'NET_AMT3', NET_AMT3)
cred_2.insert(14, 'NET_AMT4', NET_AMT4)
cred_2.insert(15, 'NET_AMT5', NET_AMT5)
cred_2.insert(16, 'NET_AMT6', NET_AMT6)

In [ ]:
cred_2 = cred_2.drop(columns = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 
                       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'])


In [ ]:
cred_2

In [ ]:
cred_2 = cred_2.sample(n = 10000)
cred_2

In [ ]:
cred_2

TRAIN_TEST SPLIT + NORMALIZATION OF NUMERICAL COLUMNS

In [ ]:
y = cred_2.iloc[:, -1].to_numpy()
X = cred_2.drop(['DEFAULT/NO-DEFAULT'], axis = 1)

In [ ]:
@dataclass
class algo:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def numericalnorm(self, X_train, X_test):
        self.X_1 = self.X_train.iloc[:, 0:9] # Xtrain not normalize
        self.X_2 = self.X_train.iloc[:, 9:23] # Xtrain normalize
        self.X_11 = self.X_test.iloc[:, 0:9] # Xtest not normalize
        self.X_22 = self.X_test.iloc[:, 9:23] # Xtest normalize

        
        return self.X_1, self.X_2, self.X_11, self.X_22
#         print(X_11)
#         print(X_12)
        
    def dataSplit(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.3, random_state = 0)
#         print(X_train)
        
        return self.X_train, self.X_test, self.y_train, self.y_test
        
    def normalizeTrainX(self, X):
        
        mean = np.mean(X, 0)
        std = np.std(X, 0)
        normalized = (X - mean)/std
        return normalized, mean, std
#         print(normalized)

    def normalizeTestX(self, X, trainMean, trainStd):
        normalized = (X - trainMean)/trainStd
        return normalized
#         print(trainMean)
 
    def datafit(self, X, y):

        X_train, X_test, y_train, y_test = self.norm()


        return X_train, X_test, y_train, y_test
        
    
    def norm(self):
        X_train, X_test, y_train, y_test = self.dataSplit(self.X, self.y)
        
        sm = SMOTE(random_state = 0)
        self.X_train, self.y_train = sm.fit_resample(self.X_train, self.y_train)
        
        X_1, X_2, X_11, X_22 = self.numericalnorm(self.X_train, self.X_test)
        


        X_train, mean, std = self.normalizeTrainX(self.X_2)        
        X_train1 = pd.concat([self.X_1, X_train], axis = 1)
        
        X_test1 = self.normalizeTestX(self.X_22, mean, std)
        
        X_test1 = pd.concat([self.X_11, X_test1], axis = 1)
        
#         print(len(self.y_test))
        return X_train1, X_test1, self.y_train, y_test


        

        
        




In [ ]:
al = algo(X,y)
# al.numericalnorm()
# al.dataSplit(X_2, y)
al.datafit(X, y)

## Logistic Regression

In [ ]:
@dataclass
class LogisticRegression:
    
    def __init__(self, learningRate, tolerance, maxIteration) -> None:
        self.tolerance = tolerance
        self.learningRate = learningRate
        self.maxIteration = maxIteration
        self.d = None

  
    def sigmoid(self, z):
        sig = 1/(1+np.exp(-z))
        return sig

    def costFunction(self, X, y):
        sig = self.sigmoid(X.dot(self.w))
        # approach1
        cost = (y*np.log(sig)+(1-y)*np.log(1-sig)).sum()
        cost = -cost/X.shape[0]
        # approach2
        # cost = (np.log(np.ones(X.shape[0])+np.exp(X.dot(self.w))-X.dot(self.w).dot(y)))
        # cost = cost.sum()
        return cost

    def gradient(self, X, y):
        sig = self.sigmoid(X.dot(self.w))
        gradient = (sig-y).dot(X)
        return gradient

    def gradientDescent(self, X, y):
        errors = []
        last_error = float('inf')
        
        for i in tqdm(range(self.maxIteration)):
            self.w = self.w - self.learningRate * self.gradient(X, y)
            current_error = self.costFunction(X, y)
            diff = last_error - current_error
            last_error = current_error
            errors.append(current_error)

            if np.abs(diff) < self.tolerance:
                print('model stopped learning')
                break
        return

    def predict(self, X):
        sig = self.sigmoid(X.dot(self.w))
        return np.around(sig)               

    def evaluate(self, y, y_hat):
        y = (y==1)
        y_hat = (y_hat == 1)                
        recall = (y & y_hat).sum()/y.sum()
        precision = (y & y_hat).sum()/y_hat.sum()
        f_score = 2*(precision*recall)/(precision+recall)
        accuracy = np.sum(y == y_hat)/len(y)
        print(y_hat)
        
        df = {
        
        'Accuracy': [accuracy],
        'F1-Score':  [f_score],
        'Recall': [recall],
        'Precision': [precision]}

        self.d = pd.DataFrame(df)
        
        return f_score, recall, precision, accuracy

    def fit(self):
        X_train, X_test, y_train, y_test = al.datafit(X, y)
#         print(X_test)
        print(X_train.shape[1])
        print('Solving using GD')
        self.w = np.ones(X_train.shape[1], dtype=np.float64)*0
        self.gradientDescent(X_train, y_train)
        print(self.w)
        print(X_test.columns)
        y_test_hat = self.predict(X_test)
        f_score, recall, precision, accuracy = self.evaluate(y_test, y_test_hat)
        print('Training F1 score {}'.format(f_score))
        print('Training recall score {}'.format(recall))
        print('Training precision score {}'.format(precision))
        print('Accuray {}'.format(accuracy))
        
        
        
        
        
        fpr, tpr, _ = roc_curve(y_test, y_test_hat)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=[5,5])
        plt.plot(fpr,tpr, color = 'darkorange', lw=2, label = 'ROC curve (area = %0.4f)' % roc_auc)
        plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.legend(loc='lower right')
        plt.show()
        
        confusion_matrix = metrics.confusion_matrix(y_test, y_test_hat)
        cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

        cm_display.plot()
        plt.show()
        
        


In [ ]:
lr1fe = LogisticRegression(learningRate = 0.01, tolerance = 0.05, maxIteration = 10000)

In [ ]:
%%time
lr1fe.fit()

In [ ]:
%%time
lr2fe = LogisticRegression(learningRate = 0.001, tolerance = 0.005, maxIteration = 50000)
lr2fe.fit()

In [ ]:
%%time
lr3fe = LogisticRegression(learningRate = 0.0001, tolerance = 0.0005, maxIteration = 50000)
lr3fe.fit()

In [ ]:
%%time
lr4fe = LogisticRegression(learningRate = 0.00001, tolerance = 0.00005, maxIteration = 50000)
lr4fe.fit()

In [ ]:
%%time
lr5fe = LogisticRegression(learningRate = 0.00001, tolerance = 0.0005, maxIteration = 50000)
lr5fe.fit()

In [ ]:
l1 = pd.DataFrame(lr1fe.d)
l2 = pd.DataFrame(lr2fe.d)
l3 = pd.DataFrame(lr3fe.d)
l4 = pd.DataFrame(lr4fe.d)
l5 = pd.DataFrame(lr5fe.d)
logistic_fe = pd.concat([l1, l2, l3, l4, l5])
logistic_fe = logistic_fe.reset_index(drop = True)
logistic_fe['Learning Rate'] = [0.01, 0.001, 0.0001, 0.00001, 0.00001]
logistic_fe['Tolerance'] = [0.05, 0.005, 0.0005, 0.00005, 0.0005]
logistic_fe

In [ ]:
logistic_best_fe = pd.DataFrame(lr4fe.d)
logistic_best_fe

## Naive Bayes

In [ ]:
@dataclass

class NaiveBayes:
        X:np.matrix
        y:np.array

        def __init__(self, X, y):
            self.X = X
            self.y = y

        def dataSplit(self):
            self.X_train, self.X_test, self.y_train, self.y_test = al.datafit(y)

        def fitDistribution(self, data):
            mean = np.mean(data)
            std = np.std(data)
            dist = norm(mean, std)
            return dist

        def probabilityFunction(self, X, prior, dist1, dist2, dist3, dist4, dist5, dist6, dist7, dist8, dist9, dist10, dist11,
        dist12, dist13, dist14, dist15, dist16, dist17):
            return prior*dist1.pdf(X[0])*dist2*dist3*dist4*dist5.pdf(X[4])*dist6*dist7*dist8*dist9*dist10*dist11*dist12.pdf(X[11])*dist13.pdf(X[12])*dist14.pdf(X[13])*dist15.pdf(X[14])*dist16.pdf(X[15])*dist17.pdf(X[16])
        
        def discreteNaive(self, X, y):
            a = 1
            X_train = X
            y_train = y
            k = len(np.unique(self.X))
            if (np.where(X_train)[0].shape[0]/len(y_train))==0:
                return (np.where(X_train)[0].shape[0]+a)/(len(y_train)+a*k)
            else:
                return np.where(X_train)[0].shape[0]/len(y_train)

In [ ]:
@dataclass
class NaiveBayes:
    X:np.matrix
    y:np.array

    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.d = None

    def dataSplit(self):
        self.X_train, self.X_test, self.y_train, self.y_test = al.datafit(X, y)

    def fitDistribution(self, data):
        mean = np.mean(data)
        std = np.std(data)
        dist = norm(mean, std)
        return dist

    def probabilityFunction(self, X, prior, dist1, dist2, dist3, dist4, dist5, dist6, dist7, dist8, dist9, dist10, dist11,
                           dist12, dist13, dist14, dist15, dist16, dist17):
        return prior*dist1*dist2*dist3*dist4*dist5*dist6*dist7*dist8*dist9*dist10.pdf(X[9])*dist11.pdf(X[10])*dist12.pdf(X[11])*dist13.pdf(X[12])*dist14.pdf(X[13])*dist15.pdf(X[14])*dist16.pdf(X[15])*dist17.pdf(X[16])
        
    
    def discreteNaive(self, X, y):
        
        a = 1
        X_train = X
        y_train = y
        k = len(np.unique(self.X))
        
        if (np.where(X_train)[0].shape[0]/len(y_train))==0:
            return (np.where(X_train)[0].shape[0]+a)/(len(y_train)+a*k)
        else:
            # else return the prob
            return np.where(X_train)[0].shape[0]/len(y_train)
        

    def fit(self):
        
        self.dataSplit()

        print(self.X_train.shape, self.y_train.shape)
        self.X0_train = self.X_train[self.y_train == 0]
        self.X1_train = self.X_train[self.y_train == 1]

        self.prior_y0 = len(self.X0_train)/len(self.X_train)
        self.prior_y1 = len(self.X1_train)/len(self.X_train)
        
        #Continuous variables

        self.dist_X0_9 = self.fitDistribution(self.X0_train.values[:, 9]) #Gaussian probability for LIMIT_BAL of class 0
        self.dist_X0_10 = self.fitDistribution(self.X0_train.values[:, 10]) #Gaussian probability for AGE of class 0
        self.dist_X0_11 = self.fitDistribution(self.X0_train.values[:, 11]) #Gaussian probability for NET_AMT1 of class 0
        self.dist_X0_12 = self.fitDistribution(self.X0_train.values[:, 12]) #Gaussian probability for NET_AMT2 of class 0
        self.dist_X0_12 = self.fitDistribution(self.X0_train.values[:, 12]) #Gaussian probability for NET_AMT2 of class 0
        self.dist_X0_13 = self.fitDistribution(self.X0_train.values[:, 13]) #Gaussian probability for NET_AMT3 of class 0
        self.dist_X0_14 = self.fitDistribution(self.X0_train.values[:, 14]) #Gaussian probability for NET_AMT4 of class 0
        self.dist_X0_15 = self.fitDistribution(self.X0_train.values[:, 15]) #Gaussian probability for NET_AMT5 of class 0
        self.dist_X0_16 = self.fitDistribution(self.X0_train.values[:, 16]) #Gaussian probability for NET_AMT6 of class 0
    
        self.dist_X1_9 = self.fitDistribution(self.X1_train.values[:, 9]) #Gaussian probability for LIMIT_BAL of class 1
        self.dist_X1_10 = self.fitDistribution(self.X1_train.values[:, 10]) #Gaussian probability for AGE of class 1
        self.dist_X1_11 = self.fitDistribution(self.X1_train.values[:, 11]) #Gaussian probability for NET_AMT1 of class 1
        self.dist_X1_12 = self.fitDistribution(self.X1_train.values[:, 12]) #Gaussian probability for NET_AMT2 of class 1
        self.dist_X1_13 = self.fitDistribution(self.X1_train.values[:, 13]) #Gaussian probability for NET_AMT3 of class 1
        self.dist_X1_14 = self.fitDistribution(self.X1_train.values[:, 14]) #Gaussian probability for NET_AMT4 of class 1
        self.dist_X1_15 = self.fitDistribution(self.X1_train.values[:, 15]) #Gaussian probability for NET_AMT5 of class 1
        self.dist_X1_16 = self.fitDistribution(self.X1_train.values[:, 16]) #Gaussian probability for NET_AMT6 of class 1
        
    def predict(self):
        y_pred = []
        
        for sample, target in zip(self.X_test.values, self.y_test):
            
            py0 = self.probabilityFunction(sample, self.prior_y0, 
                                           self.discreteNaive(self.X0_train.values[:, 0]==sample[1], self.X0_train),
                                           self.discreteNaive(self.X0_train.values[:, 1]==sample[1], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 2]==sample[2], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 3]==sample[3], self.X0_train),
                                           self.discreteNaive(self.X0_train.values[:, 4]==sample[1], self.X0_train),
                                           self.discreteNaive(self.X0_train.values[:, 5]==sample[5], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 6]==sample[6], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 7]==sample[7], self.X0_train), 
                                           self.discreteNaive(self.X0_train.values[:, 8]==sample[8], self.X0_train), 
                                           self.dist_X0_9, self.dist_X0_10, 
                                           self.dist_X0_11, self.dist_X0_12, self.dist_X0_13, 
                                           self.dist_X0_14, self.dist_X0_15, self.dist_X0_16)
            py1 = self.probabilityFunction(sample, self.prior_y1, 
                                           self.discreteNaive(self.X1_train.values[:, 0]==sample[1], self.X1_train),
                                           self.discreteNaive(self.X1_train.values[:, 1]==sample[1], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 2]==sample[2], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 3]==sample[3], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 4]==sample[1], self.X1_train),
                                           self.discreteNaive(self.X1_train.values[:, 5]==sample[5], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 6]==sample[6], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 7]==sample[7], self.X1_train), 
                                           self.discreteNaive(self.X1_train.values[:, 8]==sample[8], self.X1_train), 
                                           self.dist_X1_9, self.dist_X1_10, 
                                           self.dist_X1_11, self.dist_X1_12, self.dist_X1_13, 
                                           self.dist_X1_14, self.dist_X1_15, self.dist_X1_16)
        
            y_pred.append(np.argmax([py0, py1]))
          
        y_pred = np.array(y_pred)  
        
        print(y_pred)
        
        
        accuracy = np.sum(self.y_test == y_pred)/len(self.y_test)
        precision = (self.y_test & y_pred).sum()/y_pred.sum()
        recall = (self.y_test & y_pred).sum() / self.y_test.sum()
        f_score = 2 * (precision * recall) / (precision + recall)
        
        print("Accuracy is {}".format(accuracy)) 
        print("F-score is {}".format(f_score)) 
        print("Precision is {}".format(precision))
        print("Recall is {}".format(recall))
        
        
        df = {       
        'Accuracy': [accuracy],
        'F1-Score':  [f_score],
        'Recall': [recall],
        'Precision': [precision]}

        self.d = pd.DataFrame(df)
        
        fpr, tpr, _ = roc_curve(self.y_test, y_pred)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=[5,5])
        plt.plot(fpr,tpr, color = 'darkorange', lw=2, label = 'ROC curve (area = %0.4f)' % roc_auc)
        plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.legend(loc='lower right')
        plt.show()
        
        confusion_matrix = metrics.confusion_matrix(self.y_test, y_pred)
        cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

        cm_display.plot()
        plt.show()

In [ ]:
clf = NaiveBayes(X, y)

In [ ]:
clf.fit()

In [ ]:
%%time
clf.predict()

In [ ]:
naive_fe = pd.DataFrame(clf.d)
naive_fe

## Decision Tree

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [ ]:
class DecisionTreeClassifier():
    def __init__(self, X, y, min_samples_split=2, max_depth = 2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

        self.X = X
        self.y = y

        self.flag = True
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, y):
        ''' function to compute leaf node '''
        
        y = list(y)
        return max(y, key=y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        if self.flag:
          print("Decision Tree Representation :\n")    
          self.flag=False
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def evaluate(self, X_test, y):
        y_hat = self.predict(self.X_test) 
        y_hat = np.array(y_hat).reshape(len(y_hat),1)
        # print(y_hat)
        y = (y==1)
        y_hat = (y_hat == 1)
        accuracy = np.sum(y == y_hat)/len(y)
        recall = (y & y_hat).sum()/y.sum()
        precision = (y & y_hat).sum()/y_hat.sum()
        f_score = 2*(precision*recall)/(precision+recall)
        print("Metric Report :\nF1-Score:{}\nRecall:{}\nPrecision:{}\nAccuracy:{}".format(f_score, recall, precision, accuracy))

        
        df = {
        'Accuracy': [accuracy],
        'F1-Score':  [f_score],
        'Recall': [recall],
        'Precision': [precision]}

        self.d = pd.DataFrame(df)
        fpr, tpr, _ = roc_curve(self.y_test, y_hat)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=[5,5])
        plt.plot(fpr,tpr, color = 'darkorange', lw = 2, label = 'ROC curve (area = %0.4f)' % roc_auc)
        plt.plot([0,1], [0,1], color='navy', lw = 2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.legend(loc='lower right')
        plt.show()
        
        confusion_matrix = metrics.confusion_matrix(self.y_test, y_hat)
        cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

        cm_display.plot()
        plt.show()
        
        
        
        
        
        
    def fit(self):
        ''' function to train the tree '''
        from sklearn.model_selection import train_test_split
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
        dataset = np.column_stack((self.X_train, self.y_train))
        self.root = self.build_tree(dataset)
        y_pred = self.evaluate(self.X_test, self.y_test)
        # print("Decision Tree Representation :\n")
        # self.print_tree()
#         return X_train, X_test, y_train, y_test, y_pred
    


    def info(self):
        self.fit()
        self.print_tree()
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [ ]:
# y = y.reshape(-1,1)
y

In [ ]:
X = cred_2.iloc[:, :-1].values
y = cred_2.iloc[:, -1].values.reshape(-1,1)

In [ ]:
%%time
dt5fe = DecisionTreeClassifier(X, y, min_samples_split = 8, max_depth = 4)
dt5fe.info()

In [ ]:
dec_tree5fe = pd.DataFrame(dt5fe.d)
dec_tree5fe

In [ ]:
# decision_treefe = pd.concat([dec_tree5fe])
# decision_treefe = decision_treefe.reset_index(drop = True)
# decision_treefe['Min_Samples_Split'] = [3, 4, 5, 5, 8]
# decision_treefe['Max_Depth'] = [8, 6, 5, 8, 4]
# decision_treefe

## COMPARISON

In [ ]:
analysisfe = pd.concat([logistic_best_fe, naive_fe, dec_tree5])
analysisfe = analysisfe.reset_index(drop = True)
analysisfe

In [ ]:
analysisfe['Model'] = ['Logistic', 'Naive', 'Decision Tree']
analysisfe

In [ ]:
final = pd.concat([analysis, analysisfe])
final['Feature Engineering'] = ['No', 'No', 'No', 'Yes', 'Yes', 'Yes']
final

In [ ]:
import plotly.graph_objects as go

models = ['Logistic', 'Naive Bayes', 'Decision Tree']

fig = go.Figure()
fig.add_trace(go.Bar(x = models,
                y = analysis['Accuracy'],
                name ='Accuracy',
                marker_color='salmon'
                ))
fig.add_trace(go.Bar(x = models,
                y = analysis['F1-Score'],
                name='F1-Score',
                marker_color='sandybrown'
                ))
fig.add_trace(go.Bar(x = models,
                y=analysis['Recall'],
                name='Recall',
                marker_color='saddlebrown'
                ))
fig.add_trace(go.Bar(x=models,
                y=analysis['Precision'],
                name='Precision',
                marker_color='lightsalmon'
                ))

fig.update_layout(
    title='Comparison of Machine Learning Models',
    xaxis_tickfont_size=14,
    yaxis = dict(
        title='Evaluation Scores',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x = 1.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)',
        yanchor="top",
        xanchor = "right",
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

### ACCURACY ANALYSIS

In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'Accuracy_test' :  analysisfe['Accuracy']

})

models.sort_values(by = 'Accuracy_test', ascending = False)
px.bar(data_frame = models, y = 'Accuracy_test', x = 'Model',
       color = 'Accuracy_test',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'Accuracy Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")

### F1-SCORE ANALYSIS

In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'F1-Score' :  analysisfe['F1-Score']

})

models.sort_values(by = 'F1-Score', ascending = False)
px.bar(data_frame = models, y = 'F1-Score', x = 'Model',
       color = 'F1-Score',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'F1-Score Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")


### RECALL ANALYSIS

In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'Recall' :  analysisfe['Recall']

})

models.sort_values(by = 'Recall', ascending = False)
px.bar(data_frame = models, y = 'Recall', x = 'Model',
       color = 'Recall',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'Recall Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")


### PRECISION ANALYSIS

In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'Precision' :  analysisfe['Precision']

})

models.sort_values(by = 'Precision', ascending = False)
px.bar(data_frame = models, y = 'Precision', x = 'Model',
       color = 'Precision',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'Precision Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")


In [ ]:
models = pd.DataFrame({
     'Model' : ['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
     'Accuracy_test' :  analysisfe['Accuracy']

})

models.sort_values(by = 'Accuracy_test', ascending = False)
px.bar(data_frame = models, y = 'Accuracy_test', x = 'Model',
       color = 'Accuracy_test',color_continuous_scale='haline',
       template = 'plotly_dark', title = 'Accuracy Comparison on Test Data for each Machine Learning Model').update_yaxes(categoryorder="total descending")